<a href="https://colab.research.google.com/github/AngelJmzIsd/DatasetSemaforoCovid/blob/main/GenerarMapaSemaforos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias necesarias

In [ ]:
from math import *
from PIL import Image
import numpy as np
import csv
from datetime import datetime



# Funciones previas: color y similitud

In [ ]:

#Regresa el color RGB de un pixel de una imagen en concreto
def colorPixel(nombreArchivo,x,y):
  imagen=nombreArchivo
  im = Image.open(imagen)
  pixels = im.load()
  pixel=pixels[x,y]
  return list(pixel)

#El siguiente convierte una imagen a un objeto im
def toPixels(nombreArchivo):
  imagen=nombreArchivo
  im = Image.open(imagen)
  pixels = im.load()
  return pixels


COMPARAR SIMILARIDAD DE DOS COLORES


In [ ]:


#Recibe los RGB de cada numero
def similaridad(color1, color2):
  c1=color1
  c2=color2

  r1=c1[0]
  g1=c1[1]
  b1=c1[2]

  r2=c2[0]
  g2=c2[1]
  b2=c2[2]

  a1=abs(r2-r1)
  a2=abs(g2-g1)
  a3=abs(b2-b1)

  d=sqrt((a1)^2+(a2)^2+(a3)^2)
  p=d/sqrt((255)^2+(255)^2+(255)^2)

  #Regresa el porcentaje de "Similaridad"
  return (1-p)*100

# Parámetros iniciales

In [ ]:
#colores base INICIALES - SIRVEN COMO GUIA

#VERDE
c1=[35,203,63]
#AMARILLO
c2=[249,241,20]
#NARANJA
c3=[243,128,13]
#ROJO
c4=[190,0,15]

#MINIMO


#LEER ARCHIVO CSV DE ESTADOS CODIGO Y NOMBRE
#coordenadas de los estados, codigo y nombre
datos="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/MAPAS DE PRUEBA - MAPAS COVID - copia/coordenadas.csv"

file = open(datos)
type(file)
csvreader = csv.reader(file)

#4: Extraer el header
header = []
header = next(csvreader)

#5: extraer los registros
rows = []
for row in csvreader:
        rows.append(row)

#6: Cerrar el archivo
file.close()

clave=[]
estado=[]
px=[]
py=[]

rows.reverse()
for registro in rows:
  clave.append(int(registro[0]))
  estado.append(registro[1])
  px.append(int(registro[2]))
  py.append(int(registro[3]))

# OBTENER LISTA DE LOS NOMBRES DE TODOS LOS ARCHIVOS DE IMAGEN

In [ ]:

# 1: import OS module
import os
 
# 2: Get the list of all files and directories

ruta2020="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/MAPAS DE PRUEBA - MAPAS COVID - copia/2020"
ruta2021="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/MAPAS DE PRUEBA - MAPAS COVID - copia/2021"
ruta2022="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/MAPAS DE PRUEBA - MAPAS COVID - copia/2022"

mapas2020=[]
mapas2021=[]
mapas2022=[]

path = ruta2020
mapas2020 = os.listdir(path)

path = ruta2021
mapas2021 = os.listdir(path)

path = ruta2022
mapas2022 = os.listdir(path)

# 3: prints all files


PRIMERO CON UNA SOLA IMAGEN - CREAR EL RANGO DE DATOS

In [ ]:
#Esto se supone que se hará con cada uno de los mapas

imagen="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/MAPAS DE PRUEBA - MAPAS COVID - copia/2020/15_06_2020-21_06_2020.png"
fechas=imagen[-25:-4]
fecha1= fechas[0:10]
fecha2=fechas[11:]


date1=fecha1[0:2]+"/"+fecha1[3:5]+"/"+fecha1[6:11]
date2=fecha2[0:2]+"/"+fecha2[3:5]+"/"+fecha2[6:11]

nivelSem=[]
pixels=toPixels(imagen)


for i in range (0,32):
  nivel=0
  pmin=70
  punto=pixels[px[i],py[i]]

  match1=similaridad(c1, punto)
  if match1>=pmin: nivel=1

  match2=similaridad(c2, punto)
  if match2>=pmin: nivel=2

  match3=similaridad(c3, punto)
  if match3>=pmin: nivel=3

  match4=similaridad(c4, punto)
  if match4>=pmin: nivel=4

  nivelSem.append(nivel)
  #linea=str(clave[i])+","+estado[i]+","+str(nivel)+","+date1
  #print(linea)


ESCRIBIR LOS DATOS EN LA TABLA DE INTERVALOS CSV

In [ ]:
# 1: open the file in the write mode
archivo="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/MAPAS DE PRUEBA - MAPAS COVID - copia/ResultadosMapas2.csv"
ruta="/content/drive/MyDrive/4TO SEMESTRE/DATATHON/IMAGENES DE MAPAS/ResultadosMapas"


# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
fechaHora = now.strftime("%d_%m_%Y-%H:%M:%S")
#print("date and time =", dt_string)	


nombreArchivo="IntervalosEstados-"+fechaHora+".csv"
archivo=ruta+"/"+nombreArchivo
f = open(archivo, 'w')

# 2: create the csv writer
writer = csv.writer(f)

# 3: write a row to the csv file
  #writer.writerow(row)
header2=["FechaInicio","FechaFin","ClaveEntidad","Nivel"]
writer.writerow(header2)

for i in range(0,32):
  fila=[]
  fila=[date1,date2,clave[i],estado[i],nivelSem[i]]
  writer.writerow(fila)

# 4: close the file
f.close()

# **GENERAR CADA UNA DE LAS FECHAS CON CADA ESTADO Y SU NIVEL DE RIESGO**

para eso se necesitan los datos de cada uno de los mapas, y se comienzan a agregar los datos correspondientes a las fechas de inicio y de fin

O sea, para cada día del año, colocar el nivel de cada uno de los 32 estados

In [ ]:
#Recorrer todas las fechas desde el 2020 hasta el día de hoy